In [ ]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-path-finder", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

In [ ]:
%%send_to_spark -i username -t str -n username

In [ ]:
import pyspark.sql.functions as F

## Import data from HDFS

In [ ]:
walking_edges = spark.read.parquet('/user/%s/final/parquet/walking_edges' % username)

In [ ]:
transport_edges = spark.read.parquet('/user/%s/final/parquet/transport_edges' % username)

In [ ]:
transport_edges.columns

## Shortest Path

In [ ]:
print('hello')

In [ ]:
inward_walking_edges = walking_edges.groupby('end_vertex').agg({'start_vertex': 'collect_set'})\
                                                              .withColumnRenamed('collect_set(start_vertex)', 'start_vertices')\
                                                              .toPandas().set_index('end_vertex').to_dict()['start_vertices']
walking_edge_duration = walking_edges.toPandas()
walking_edge_duration['key'] = walking_edge_duration.apply(lambda x: (x['start_vertex'], x['end_vertex']), axis=1)
walking_edge_duration = walking_edge_duration.set_index('key').to_dict()['duration']

In [ ]:
edges = transport_edges.filter('weekday == "monday"').sort(F.col('start_time').desc())

In [ ]:
n = edges.toPandas().to_numpy()

In [ ]:
i=0
for e in n:
    print(e.shape)
    i += 1
    if i > 5:
        break

In [ ]:
# This function assumes a (spark) dataframe with columns 'start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id' 
# edges is a dataframe that represents the edges of the graph, 
# edges should be sorted by order of descending starting time
# inward_walking_edges is a map route_id -> list(route_id), representing the nodes from which we can walk to a given node
# walking_edge_duration is a map (route_id, route_id) -> float, representing the duration of the walk
# end_time should be in the format 'HH:MM'
def latest_departure_paths(edges, end_stop, end_time, inward_walking_edges, walking_edge_duration):
    
    # This function computes whether edge (u, v, t, dur, route_id) can be taken
    # returns update to node u if possible, and None otherwise
    def edge_valid(u, v, time, dur, route_id):
        
        cur_best = next_transfer.get(u)
        next_edge = next_transfer.get(v)
        
        if next_edge: # needs to be defined
            time_v, next_v, route_id_v, dur_v = next_edge
            
            if route_id == 'walking':
                if  route_id_v == 'walking' and dur_v + dur <= max_walking_time:
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, next_v, route_id, dur + dur_v)
                elif route_id_v != 'walking':
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, v, route_id, dur)
                      
            
            elif (route_id == route_id_v or v == end_stop) and time + duration <= time_v: # same route or last_stop
                if (not cur_best) or time > cur_best[0]: # can we improve
                    return (time, next_v, route_id, dur + dur_v)
            
            
            elif time + duration + transfer_time <= time_v: # transfer at node_v
                if (not cur_best) or time > cur_best[0]:
                    return (time, v, route_id, dur) 
        
        # In any other case no update can be made
        return None
                
    def time_to_minutes(timestamp):
        return int(timestamp[:2]) * 60 + int(timestamp[3:5])
    
    transfer_time = 2 # at least 2 minutes to transfer
    max_walking_time = 10

    next_transfer = {} # stores for each node u the node v 
                       # where the next transfer happens format (departure_time, v, route_id to v, duration)
    end_time = time_to_minutes(end_time)
    start_time = end_time - 120 # look only at edges departing at most 2 hours before end_time
    next_transfer[end_stop] = (end_time, end_stop, None, 0)
    
    edges = edges.filter(F.col('start_time').between(start_time, end_time)).toPandas()

    for _, row in edges.iterrows():
        
        start_vertex, end_vertex, start, duration, route_id = row.start_vertex, row.end_vertex, row.start_time, row.duration, row.route_id
        
        update = edge_valid(start_vertex, end_vertex, start, duration, route_id)
        if update:
            next_transfer[start_vertex] = update
                    
            w_start_vertex = start_vertex
            for w_end_vertex in inward_walking_edges.get(w_start_vertex, []):
                duration = walking_edge_duration[(w_start_vertex, w_end_vertex)]
                w_start = start 
                update = edge_valid(w_start_vertex, w_end_vertex, w_start, duration, 'walking')
                if update:
                     next_transfer[w_start_vertex] = update
                    
        
    return next_transfer

In [ ]:
import time
start = time.time()
next_transfer = latest_departure_paths(edges, '8591221', '20:00', inward_walking_edges, walking_edge_duration)
end = time.time() 
print(end - start)

In [ ]:
def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)
    
def reconstruct_route(next_transfer, start_stop):
    time, next_stop, route_id, duration = next_transfer[start_stop]
    journey = ''
    while next_stop != start_stop:
        journey += """  \n
                        %s : %s 
                        | 
                        | %s (%s min) 
                        | 
                        %s : %s """ % (minutes_to_timestamp(time),\
                                       stop_id_to_stop[start_stop],\
                                       route_id_to_route_name[route_id],\
                                       duration, minutes_to_timestamp(time + duration), stop_id_to_stop[next_stop])
        start_stop = next_stop
        time, next_stop, route_id, duration = next_transfer[next_stop]
        
    return journey

In [ ]:
route_id_to_route_name = routes.select('route_id', 'route_desc', 'route_short_name').toPandas()
route_id_to_route_name['route_name'] = route_id_to_route_name['route_desc'] + ' ' + route_id_to_route_name['route_short_name']
route_id_to_route_name = route_id_to_route_name.set_index('route_id').to_dict()['route_name']
route_id_to_route_name['walking'] = 'walk'

In [ ]:
stop_id_to_stop = stops.select('stop_id', 'stop_name').toPandas().set_index('stop_id').to_dict()['stop_name']

In [ ]:
def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)
    
def reconstruct_route(next_transfer, start_stop):
    time, next_stop, route_id, duration = next_transfer[start_stop]
    journey = ''
    while next_stop != start_stop:
        journey += """  \n
                        %s : %s 
                        | 
                        | %s (%s min) 
                        | 
                        %s : %s """ % (minutes_to_timestamp(time),\
                                       stop_id_to_stop[start_stop],\
                                       route_id_to_route_name[route_id],\
                                       duration, minutes_to_timestamp(time + duration), stop_id_to_stop[next_stop])
        start_stop = next_stop
        time, next_stop, route_id, duration = next_transfer[next_stop]
        
    return journey